## Hybrid 検索
```
クエリは、VectorSearchとBM25Searchで並行して行い、RRFでランクを統合する  
Query -+- embedding - VectorSearch -+-> RRF   
       +- Tokenize  - BM25Search   -+   
```

### ライブラリ/モデル設定/クライアント設定

In [1]:
import re, sys
import numpy as np
import pandas as pd
pd.set_option('display.width', 150)
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows',  None)
from pprint import pprint

import google.genai as genai # LLM , embedding
import cohere                # rerank # 未使用
import chromadb              # Vector Store
import langchain_core
from langchain_core.documents import Document

print(f"Python: {sys.version}")
print(f"genai={genai.__version__}, cohere={cohere.__version__}," +\
      f"chromadb={chromadb.__version__}, langchain_core={langchain_core.__version__}, ")

Python: 3.13.9 (tags/v3.13.9:8183fa5, Oct 14 2025, 14:09:13) [MSC v.1944 64 bit (AMD64)]
genai=1.52.0, cohere=5.20.0,chromadb=1.3.5, langchain_core=1.1.0, 


In [2]:
# Geminiモデルを指定
GEMINI_EMBEDDING_MODEL = 'gemini-embedding-001'
#GEMINI_LLM_MODEL      = 'gemini-2.0-flash' # Limit-Rateが高い
GEMINI_LLM_MODEL      = 'gemini-2.5-flash'  # それなりに賢い
#GEMINI_LLM_MODEL      = 'gemini-2.5-pro'
#GEMINI_LLM_MODEL      = 'gemini-3-pro'     # 未提供

# OpenAIモデルを指定
OPENAI_EMBEDDING_MODEL = 'text-embedding-3-small'

# Cohereモデルを指定
#COHERE_EMBEDDING_MODEL = 'embed-v4.0'
COHERE_LLM_MODEL       = 'command-a-03-2025'
COHERE_RERANK_MODEL     = 'rerank-v3.5'

key_path = './keys/'

# Geminiクライアントを作成
with open(key_path + 'Google_API_KEY.txt', 'r') as f:
    api_key = f.read().strip()
GEMINI_CLIENT = genai.Client(api_key=api_key)

# Cohereクライアントを作成
with open(key_path + 'Cohere_API_KEY.txt', 'r') as f:
    api_key = f.read().strip()
COHERE_CLIENT = cohere.ClientV2(api_key=api_key)

# Chromaクライアントを作成
CHROMA_CLIENT = chromadb.EphemeralClient()  # インメモリで作成


### 文の整形関数

トークン化する前に、省略部分の補完、不要な部分を削除

In [3]:
def restructure_text(text):
    # プロンプトを定義
    prompt = '''
以下の文書を次のルールで編集してください。編集結果のみを回答してください。
1) 複数の行にわたる文を、一行の文にしてください。
2) 文が、項番や見出し記号で始まる場合には、項番、見出し記号部分を削除してください。
3) 文の主語、目的語が省略されている場合には、直前の文の主語、目的語で補完した文としてください。
   ただし実験者/作業者を主語の場合には省略することとします。
4) 項番や見出し記号と、見出しの単語や複合語だけから構成される行は削除してください。
'''
    system_instruction = 'あなたは日本語文法、意味解析の専門家です'
    prompt += '\n\n'+text
    temperature = 0.2
    
    # モデルを使用してレスポンスを生成
    response = GEMINI_CLIENT.models.generate_content(
        model=GEMINI_LLM_MODEL, 
        contents=[prompt],  # リスト形式
        config=genai.types.GenerateContentConfig(
            system_instruction=system_instruction,
            temperature=temperature
        )
    ).text
    return response

### 日本語 Tokenizer

入力文を、形態素解析エンジンでトークン化、必要な品詞のみのトークンのリストを返す  

In [4]:
### 蛇の目
# pip install janome
from janome.tokenizer import Tokenizer
tokenizer = Tokenizer()

def tokenize_and_filter_by_pos(text, target_pos=['名詞','動詞','形容詞','副詞','接頭詞']):
    # 入力文から指定された品詞のトークンを抽出
    tokens = list(tokenizer.tokenize(text))
    #pprint([(t.surface, t.part_of_speech) for t in tokens])
    parts_of_speech = [t.part_of_speech.split(',') for t in tokens]           # 品詞情報を抽出
    is_target_pos = [bool(set(p) & set(target_pos)) for p in parts_of_speech] # 必要品詞で抽出
    #pprint(is_target_pos)
    extracted_tokens = []
    extracted_tokens = [t.surface for (t, f) in zip(tokens, is_target_pos) if f]
    return extracted_tokens

pprint(tokenize_and_filter_by_pos('① 今日は、良い天気です。'))

['①', '今日', '良い', '天気']


In [5]:
### すだち
from sudachipy import Dictionary, SplitMode
tokenizer = Dictionary().create()

def tokenize_and_filter_by_pos(text, target_pos=['名詞','動詞','形容詞','副詞','接頭詞']):
    # 入力文から指定された品詞のトークンを抽出
    tokens = list(tokenizer.tokenize(text))
    #pprint([(t.surface(), t.part_of_speech()) for t in tokens])
    parts_of_speech = [t.part_of_speech() for t in tokens]                             # 品詞情報を抽出
    is_target_pos = [bool(set(p) & set(target_pos)) and 
                     not (bool(set(p) & set(['動詞'])) and
                          bool(set(p) & set(['非自立', '非自立可能'])))
                                                             for p in parts_of_speech] # 必要品詞で抽出
    #pprint(is_target_pos)
    extracted_tokens = []
    extracted_tokens = [t.surface() for (t, f) in zip(tokens, is_target_pos) if f]
    return extracted_tokens

pprint(tokenize_and_filter_by_pos('① 今日は、良い天気です。'))

['①', '今日', '良い', '天気']


## Hybrid 検索

### データ

In [6]:
# 検索対象データ
texts = [
    ('ID01', '私の趣味はサッカー観戦とプログラミングです。'),
    ('ID02', '明日の東京の天気は晴れ時々曇りでしょう。'),
    ('ID03', 'Pythonは機械学習やデータ分析で人気のある言語です。'),
    ('ID04', 'サッカーのワールドカップは4年に1度開催されます。'),
    ('ID05', '昨日は雨が降っていましたが、今日は良い天気です。')
]
texts_df = pd.DataFrame(texts, columns=['source','doc'])
display(texts_df)

# 検索したい文 (クエリ)
query_text = 'Pythonのプログラミングやデータ分析について'

,source,doc
0,ID01,私の趣味はサッカー観戦とプログラミングです。
1,ID02,明日の東京の天気は晴れ時々曇りでしょう。
2,ID03,Pythonは機械学習やデータ分析で人気のある言語です。
3,ID04,サッカーのワールドカップは4年に1度開催されます。
4,ID05,昨日は雨が降っていましたが、今日は良い天気です。


### BM25

In [7]:
### BM25
# pip install rank-bm25
from rank_bm25 import BM25Okapi, BM25Plus

tokenized_texts = [tokenize_and_filter_by_pos(d) for d in texts_df['doc']]
print(tokenized_texts)

# BM25インデックス作成、rank_bm25では一括作成のみ
bm25 = BM25Okapi(tokenized_texts)

print("BM25インデックスの準備完了。トークン化された文書数:", len(tokenized_texts))

## 検索 ====================
# (1) 検索する文を日本語トークナイザーでトークン化
tokenized_query = tokenize_and_filter_by_pos(query_text)
print(tokenized_query)

# (2) トークン列でインデックスを検索（スコアリング）
# スコアはコーパス内の各文書に対する関連度（BM25スコア）です
bm25_scores = bm25.get_scores(tokenized_query)

texts_df['bm25score'] = bm25_scores
display(texts_df.sort_values('bm25score', ascending=False))

[['趣味', 'サッカー', '観戦', 'プログラミング'], ['明日', '東京', '天気', '晴れ', '時々', '曇り'], ['Python', '機械学習', 'データ分析', '人気', '言語'], ['サッカー', 'ワールドカップ', '4', '年', '1', '度', '開催'], ['昨日', '雨', '降っ', '今日', '良い', '天気']]
BM25インデックスの準備完了。トークン化された文書数: 5
['Python', 'プログラミング', 'データ分析', 'つい']


,source,doc,bm25score
2,ID03,Pythonは機械学習やデータ分析で人気のある言語です。,2.308529
0,ID01,私の趣味はサッカー観戦とプログラミングです。,1.260703
1,ID02,明日の東京の天気は晴れ時々曇りでしょう。,0.000000
3,ID04,サッカーのワールドカップは4年に1度開催されます。,0.000000
4,ID05,昨日は雨が降っていましたが、今日は良い天気です。,0.000000


### VectorSearch

In [8]:
### Retrive

# Geminiクライアントを作成
with open('keys/Google_API_KEY.txt', 'r') as f:
    api_key = f.read().strip()
GEMINI_CLIENT = genai.Client(api_key=api_key)

# Chromaクライアントを作成
CHROMA_CLIENT = chromadb.EphemeralClient()  # インメモリで作成

## Vector Storeの作成 ===========
collection_name = 'anonymous'
try:
    CHROMA_CLIENT.delete_collection(collection_name)
except:
    pass
collection = CHROMA_CLIENT.create_collection(
    name = collection_name,
    metadata={'hnsw:space': 'cosine'}  # 距離メトリック = 'cosine'
)

page_contents = []
metadatas = []
ids = []
for i, row in texts_df.iterrows(): # text毎に
    page_contents.append(row['doc'])
    metadatas.append({'source': row['source']})
    ids.append(f"doc{row['source']}")    # id: ユニークな文字列

# Embeddingの取得
response = GEMINI_CLIENT.models.embed_content(
    model=GEMINI_EMBEDDING_MODEL,
    contents=page_contents
).embeddings
doc_embs = [e.values for e in response]

# ChromaDBへ一括で追加
collection.add(
    ids=ids,
    embeddings=doc_embs,
    documents=page_contents,
    metadatas=metadatas
)
print(f"\n## DBに追加されたベクトル数: {collection.count()}")


## Queryの実行 ===========
k = 4
print(f"k={k}, query={query_text}")
# --- クエリをembedding ---
query_emb = GEMINI_CLIENT.models.embed_content(
    model=GEMINI_EMBEDDING_MODEL,
    contents=query_text
).embeddings[0].values

# ChromaDBで類似検索
results = collection.query(
    query_embeddings=query_emb,
    n_results = k,
    include = ['documents', 'metadatas', 'distances']
)
retreaved_sources = [m['source'] for m in results['metadatas'][0]]
retreaved_dists   = results['distances'][0]
retreaved_df = pd.DataFrame({
    'source':   retreaved_sources,
    'distance': retreaved_dists
})
texts_df = pd.merge(texts_df,retreaved_df,on='source',how='left')
display(retreaved_df)
display(texts_df)


## DBに追加されたベクトル数: 5
k=4, query=Pythonのプログラミングやデータ分析について


,source,distance
0,ID03,0.157276
1,ID01,0.307122
2,ID04,0.452467
3,ID05,0.458925


,source,doc,bm25score,distance
0,ID01,私の趣味はサッカー観戦とプログラミングです。,1.260703,0.307122
1,ID02,明日の東京の天気は晴れ時々曇りでしょう。,0.000000,NaN
2,ID03,Pythonは機械学習やデータ分析で人気のある言語です。,2.308529,0.157276
3,ID04,サッカーのワールドカップは4年に1度開催されます。,0.000000,0.452467
4,ID05,昨日は雨が降っていましたが、今日は良い天気です。,0.000000,0.458925


### RRF (Reciprocal Rank Fusion)

順位同士のデータを統合し、総合スコア(順位)を算出  
上位の順位ほど大きなボーナスポイントが付く  

In [9]:
from collections import defaultdict
RRF_K = 60 # RRFの平滑化定数

def weighted_reciprocal_rank_fusion(
    rankings: list[list[str]],  # 複数のランキングIDリスト (例: [bm25_results, vector_results])
    weights:  list[float]       # 各ランキングの重み (例: [0.6, 0.4])
) -> list[tuple[str, float]]:
    """
    二組のランキング情報とその重みを受け取り、RRFで統合スコアを計算する関数 (Weighted RRF)。
    
    Args:
        rankings: 各リトリーバーから得られた文書IDのランキングリストのリスト。
                  [['ID3','ID2','ID1'], ['ID3','ID2','ID1']]
        weights: 各ランキングに対応する重みのリスト。
                  [0.6, 0.4]        
    Returns:
        文書IDと最終スコアのタプルを要素とする、スコア降順のリスト。
    """
    
    if len(rankings) != len(weights):
        raise ValueError("ランキングの数と重みの数が一致しません。")
        
    fused_scores: Dict[str, float] = defaultdict(float)
    
    # 1. 各リトリーバーのランキングをループし、スコアを計算
    for i, rank_list in enumerate(rankings):
        weight = weights[i]
        
        # リスト内の各文書とその順位をループ
        for rank, doc_id in enumerate(rank_list):
            # 順位は0から始まるため、r_i は rank + 1
            r_i = rank + 1 
            
            # Weighted RRFスコアを計算し、合計する
            # S_d = w_i * [ 1 / (r_i + k) ]
            score_contribution = weight * (1 / (r_i + RRF_K))
            fused_scores[doc_id] += score_contribution

    # 2. スコアが高い順にソート
    # 結果をタプル (文書ID, スコア) のリストとして返す
    sorted_ranking = sorted(
        fused_scores.items(), key=lambda item: item[1], reverse=True
    )
    return sorted_ranking

# ----------------------------------------------------
# テスト

# 検索結果の定義 (文書IDはユニークな文字列)
test_df = pd.DataFrame({
    'source'  : ['Doc_A', 'Doc_B', 'Doc_C', 'Doc_D', 'Doc_E', 'Doc_F'], 
    'bm25_score':  [8, 3, 5, 2, 1, np.nan], # キーワード検索の結果
    'vector_dist': [2, 1, 3, 6, np.nan, 9]  # ベクトル検索の結果(distination)
})
bm25_results   = test_df.dropna(subset=['bm25_score'] ).sort_values('bm25_score',  ascending=False)['source'].dropna()
vector_results = test_df.dropna(subset=['vector_dist']).sort_values('vector_dist', ascending=True )['source'].dropna()
# 重み: 前者(BM25/キーワード一致) を重視する設定 (合計1.0になるように設定)
weights = [0.6, 0.4]

display(test_df)
print(bm25_results)
print(vector_results)
print(f"weights={weights}")

# RRFの実行 ------------------------------------------------------------
all_rankings = [bm25_results, vector_results]
hybrid_ranking = weighted_reciprocal_rank_fusion(all_rankings, weights)
# ---------------------------------------------------------------------

pprint(hybrid_ranking)
rrf_result_df = pd.DataFrame(hybrid_ranking, columns=['source', 'rrf_score'])
test_df =  pd.merge(test_df, rrf_result_df, on='source', how='left')
display(test_df)

,source,bm25_score,vector_dist
0,Doc_A,8.0,2.0
1,Doc_B,3.0,1.0
2,Doc_C,5.0,3.0
3,Doc_D,2.0,6.0
4,Doc_E,1.0,NaN
5,Doc_F,NaN,9.0


0    Doc_A
2    Doc_C
1    Doc_B
3    Doc_D
4    Doc_E
Name: source, dtype: object
1    Doc_B
0    Doc_A
2    Doc_C
3    Doc_D
5    Doc_F
Name: source, dtype: object
weights=[0.6, 0.4]
[('Doc_A', 0.0162876784769963),
 ('Doc_B', 0.01608118657298985),
 ('Doc_C', 0.016026625704045058),
 ('Doc_D', 0.015625),
 ('Doc_E', 0.009230769230769232),
 ('Doc_F', 0.006153846153846155)]


,source,bm25_score,vector_dist,rrf_score
0,Doc_A,8.0,2.0,0.016288
1,Doc_B,3.0,1.0,0.016081
2,Doc_C,5.0,3.0,0.016027
3,Doc_D,2.0,6.0,0.015625
4,Doc_E,1.0,NaN,0.009231
5,Doc_F,NaN,9.0,0.006154


### Hybrid検索結果

In [10]:
# 順位データの作成
bm25_results   = texts_df.dropna(subset=['bm25score']  ).sort_values('bm25score', ascending=False)['source'].dropna()
vector_results = texts_df.dropna(subset=['distance']   ).sort_values('distance',  ascending=True )['source'].dropna()
# 重み: 前者(BM25/キーワード一致) を重視する設定 (合計1.0になるように設定)
weights = [0.6, 0.4]

display(texts_df)
print(f"bm25_results:\n{bm25_results}")
print(f"vector_results:\n{vector_results}")
print(f"weights={weights}")

# RRFの実行 ------------------------------------------------------------
all_rankings = [bm25_results, vector_results]
hybrid_ranking = weighted_reciprocal_rank_fusion(all_rankings, weights)
# ---------------------------------------------------------------------

print(f"\nhybrid_ranking result:")
pprint(hybrid_ranking)
rrf_result_df = pd.DataFrame(hybrid_ranking, columns=['source', 'rrf_score'])
textsx_df =  pd.merge(texts_df, rrf_result_df, on='source', how='left')
display(textsx_df.sort_values('rrf_score', ascending=False))

,source,doc,bm25score,distance
0,ID01,私の趣味はサッカー観戦とプログラミングです。,1.260703,0.307122
1,ID02,明日の東京の天気は晴れ時々曇りでしょう。,0.000000,NaN
2,ID03,Pythonは機械学習やデータ分析で人気のある言語です。,2.308529,0.157276
3,ID04,サッカーのワールドカップは4年に1度開催されます。,0.000000,0.452467
4,ID05,昨日は雨が降っていましたが、今日は良い天気です。,0.000000,0.458925


bm25_results:
2    ID03
0    ID01
1    ID02
3    ID04
4    ID05
Name: source, dtype: object
vector_results:
2    ID03
0    ID01
3    ID04
4    ID05
Name: source, dtype: object
weights=[0.6, 0.4]

hybrid_ranking result:
[('ID03', 0.01639344262295082),
 ('ID01', 0.016129032258064516),
 ('ID04', 0.01572420634920635),
 ('ID05', 0.015480769230769232),
 ('ID02', 0.009523809523809523)]


,source,doc,bm25score,distance,rrf_score
2,ID03,Pythonは機械学習やデータ分析で人気のある言語です。,2.308529,0.157276,0.016393
0,ID01,私の趣味はサッカー観戦とプログラミングです。,1.260703,0.307122,0.016129
3,ID04,サッカーのワールドカップは4年に1度開催されます。,0.000000,0.452467,0.015724
4,ID05,昨日は雨が降っていましたが、今日は良い天気です。,0.000000,0.458925,0.015481
1,ID02,明日の東京の天気は晴れ時々曇りでしょう。,0.000000,NaN,0.009524


## Langchain EnsembleRetriever

Langchainを使って

In [11]:
import os
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_chroma import Chroma
from langchain_community.retrievers import BM25Retriever
from langchain_classic.retrievers.ensemble import EnsembleRetriever
from langchain_core.documents import Document

# embeddingに使うAPIキーの設定
with open('keys/Google_API_KEY.txt', 'r') as f:
    os.environ["GOOGLE_API_KEY"] = f.read().strip()

# --- 1. ドキュメントの準備 ---
docs_text = [
    "私の趣味はサッカー観戦とプログラミングです。",
    "明日の東京の天気は晴れ時々曇りでしょう。",
    "Pythonは機械学習やデータ分析で人気のある言語です。",
    "サッカーのワールドカップは4年に1度開催されます。",
    "昨日は雨が降っていましたが、今日は良い天気です。"
]
# LangChainのDocument形式に変換
documents = [Document(page_content=t) for t in docs_text]

# --- 2. Vector Retriever (意味検索) の作成 ---
# Embeddingモデルの定義
embeddings = GoogleGenerativeAIEmbeddings(model='models/embedding-001')

# ChromaDBを作成し、Retrieverとして機能させる
vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embeddings,
    collection_name='hybrid_test_collection'
)

# vector_retriever: ベクトル検索を行う部品（上位2件取得）
vector_retriever = vectorstore.as_retriever(search_kwargs={'k': 4})

# --- 3. Keyword Retriever (BM25) の作成 ---
# bm25_retriever: キーワード検索を行う部品（上位2件取得）
bm25_retriever = BM25Retriever.from_documents(documents)
bm25_retriever.k = 4

# --- 4. Hybrid Retriever (Ensemble) の作成 ---
# EnsembleRetrieverが内部で「Reciprocal Rank Fusion (RRF)」を行い、結果を統合します
ensemble_retriever = EnsembleRetriever(
    retrievers=[vector_retriever, bm25_retriever],
    weights=[0.6, 0.4]  # ベクトル検索とキーワード検索の重み付け (合計1.0にしなくても動作はしますが、比率として機能します)
)

# --- 5. 実行 ---
query = 'Pythonのプログラミングやデータ分析について'
results = ensemble_retriever.invoke(query)

print(f"Query: {query}\n")
for i, doc in enumerate(results, start=1):
    print(f"{i}. {doc.page_content}")

Query: Pythonのプログラミングやデータ分析について

1. Pythonは機械学習やデータ分析で人気のある言語です。
2. サッカーのワールドカップは4年に1度開催されます。
3. 昨日は雨が降っていましたが、今日は良い天気です。
4. 明日の東京の天気は晴れ時々曇りでしょう。
